In [1]:
# Import required libraries

import GetOldTweets3 as got
import datetime 
import time as ti
import pickle
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 

  
stop_words = set(stopwords.words('english'))            # Set of English stopwords
lemmatizer = WordNetLemmatizer()                        # To lemmatize the data

model = pickle.load(open("Sentiment Predictor", 'rb'))  # Import the sentiment predicting model


In [2]:
def text_process(tweet):
    
    processed_tweet = [] # To store processed text
    
    tweet = tweet.lower() # Convert to lower case

    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', 'URL', tweet) # Replaces any URLs with the word URL
    
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet) # Replace @handle with the word USER_MENTION
    
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet) # Removes # from hashtag
    
    tweet = re.sub(r'\brt\b', '', tweet) # Remove RT (retweet)
    
    tweet = re.sub(r'\.{2,}', ' ', tweet) # Replace 2+ dots with space
    
    tweet = tweet.strip(' "\'') # Strip space, " and ' from tweet
    
    tweet = re.sub(r'\s+', ' ', tweet) # Replace multiple spaces with a single space
    
    words = tweet.split()
    
    for word in words:
            
        word = word.strip('\'"?!,.():;') # Remove Punctuations
        
        word = re.sub(r'(.)\1+', r'\1\1', word) # Convert more than 2 letter repetitions to 2 letter (happppy -> happy)
        
        word = re.sub(r'(-|\')', '', word) # Remove - & '
        
        if (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None): # Check if the word starts with an english letter
            
            if(word not in stop_words):                                 # Check if the word is a stopword.
                
                word = str(lemmatizer.lemmatize(word))                  # Lemmatize the word
                
                processed_tweet.append(word)
                
    return ' '.join(processed_tweet)

In [9]:
# Print the tweets and their sentiments continuously with an interval of 3 seconds per tweet


while(True):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('corona')\
                                               .setNear("Nagpur,India")\
                                               .setLang('en')\
                                               .setWithin("1500km")\
                                               .setMaxTweets(5)  
    
    # Criteria for collecting corona related indian tweets

    for tweets in range(5):

        tweet = got.manager.TweetManager.getTweets(tweetCriteria)[tweets]

        correct_time = tweet.date + datetime.timedelta(hours=5,minutes=30) # Convert time to IST
        correct_time = correct_time.strftime("%d-%m-%Y %I:%M:%S")

        date,time = correct_time.split()

        predict = model.predict([text_process(tweet.text)])                # Predict Sentiment Value
 
        result = "Positive" if predict>0 else ("Neutral" if predict == 0 else "Negative")

        print("Date: {} Time: {} Text: {} Sentiment: {}\n".format(date,time,tweet.text,result))

        ti.sleep(3)  # Sleep for 3 seconds - As a rate limiter (Otherwise it will print continuously)


Date: 13-07-2020 Time: 11:23:33 Text: Corona stopped many Bill's including #freetemple &amp; #anticonversions &amp; #commoncivilcode &amp; #populationcontrolbill.hope soon these will pass in both Houses with full majority Sentiment: Neutral

Date: 13-07-2020 Time: 11:23:20 Text: I see people shaking hands and hugging each other in movies now and think wow pre corona times Sentiment: Neutral

Date: 13-07-2020 Time: 11:23:20 Text: I see people shaking hands and hugging each other in movies now and think wow pre corona times Sentiment: Neutral

Date: 13-07-2020 Time: 11:23:18 Text: Hehe loved it...The answer for this question Eppudu vastundi is also a question Eppudu potundi ee Corona?  Sentiment: Positive

Date: 13-07-2020 Time: 11:23:10 Text: Good ones don’t seize to come even in Corona times........... Surprised that Geh *lot* and Pi *lot* aren't getting along . . . They have a *LOT* in common  Sentiment: Positive

Date: 13-07-2020 Time: 11:23:03 Text: Shit! I seriously read it as "Cor